In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
import torch.nn.init as init
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 5000
N_bound = 200

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
from scipy.io import loadmat

data = loadmat("usol_D_0.001_k_5.mat")

t_exact = data["t"].reshape(-1,1)

x_exact = data["x"].reshape(-1,1)

Exact_u = data["u"]

print('t_exact:',t_exact.shape)
print('x_exact:',x_exact.shape)
print('Exact_u:',Exact_u.shape)

In [ ]:
X_exact, T_exact = np.meshgrid(x_exact,t_exact)
print('X_exact:',X_exact.shape)
print('T_exact:',T_exact.shape)

X_exact_flatten = X_exact.flatten()[:,None]
T_exact_flatten = T_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None] 

In [ ]:
data_star.shape

In [ ]:
random_seed = 1234
np.random.seed(random_seed)
number_observe_data = 50

index_x = (lhs(1,number_observe_data))*len(x_exact)
index_x = np.floor(index_x).reshape(number_observe_data,).astype(int)
observe_x = x_exact[index_x]

index_t = (lhs(1,number_observe_data))*len(t_exact)
index_t = np.floor(index_t).reshape(number_observe_data,).astype(int)
observe_t = t_exact[index_t]

observe_data = np.hstack((observe_x,observe_t))
observe_clear_u = Exact_u[index_t,index_x].reshape(-1,1)

############# N(0,0.1^2) #############
noise_nu = 0
noise_std = 0.1
noise_u = np.random.normal(loc=noise_nu, scale=noise_std, size=observe_clear_u.shape)
observe_u = observe_clear_u + noise_u
############# N(0,0.1^2) #############

observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_clear_u = numpy_to_tensor(observe_clear_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

In [ ]:
np.random.seed(5678)
n_test_data = 10000

index_x = (lhs(1,n_test_data))*len(x_exact)
index_x = np.floor(index_x).reshape(n_test_data,).astype(int)
observe_x = x_exact[index_x]

index_t = (lhs(1,n_test_data))*len(t_exact)
index_t = np.floor(index_t).reshape(n_test_data,).astype(int)
observe_t = t_exact[index_t]

test_data = np.hstack((observe_x,observe_t))
test_u = Exact_u[index_t,index_x].reshape(-1,1)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

In [8]:
C1 = torch.tensor(0.5, requires_grad=True)

In [ ]:
print('J:',len(observe_u))

In [10]:
def output_transform(data_input, u_input):
    
    x_in = data_input[:,0:1]
    t_in = data_input[:,1:2]
    
    return x_in**2 * torch.cos(torch.tensor(np.pi) * x_in) + t_in * (1 - x_in**2) * u_input

In [11]:
def get_loss_f(x_grad, y_grad, PINNs, C, return_sequence='not'):

    E_inside = PINNs(torch.cat((x_grad,y_grad),axis=1))
    E_inside = output_transform(torch.cat((x_grad,y_grad),axis=1),E_inside)
    
    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])
    E_xxx = compute_higher_order_derivatives(E_xx, [x_grad])
    E_xxt = compute_higher_order_derivatives(E_xx, [y_grad])
    E_t = compute_higher_order_derivatives(E_inside, [y_grad])
    E_tx = compute_higher_order_derivatives(E_t, [x_grad])
    E_tt = compute_higher_order_derivatives(E_t, [y_grad])

    loss_term = E_t - C * E_xx - 5 * (E_inside - E_inside**3)

    if return_sequence == 'yes':
        return torch.square(loss_term)
    else:
        return torch.mean(torch.square(loss_term))

In [12]:
def get_loss_bound(bound_x, bound_t, PINNs, C, return_sequence='not'):

    E_bound_x_zero = PINNs(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1)) 
    Exact_x_zero = bound_x**2*torch.cos(torch.tensor(np.pi)*bound_x)
    loss_bound_for_a = torch.mean(torch.square(E_bound_x_zero-Exact_x_zero))
    
    E_bound_fu_1_t = PINNs(torch.cat((-torch.ones_like(bound_t),bound_t),axis=1)) 
    Exact_fu_1_t = -torch.ones_like(E_bound_fu_1_t)
    loss_bound_for_b = torch.mean(torch.square(E_bound_fu_1_t-Exact_fu_1_t))
    
    E_bound_1_t = PINNs(torch.cat((torch.ones_like(bound_t),bound_t),axis=1))
    Exact_1_t = -torch.ones_like(E_bound_1_t)
    loss_bound_for_c = torch.mean(torch.square(E_bound_1_t-Exact_1_t))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c
    
    return loss_bound_value

In [13]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [14]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[64]*6, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [15]:
x_inside_all = torch.cat((x_inside,observe_data[:,0:1]),axis=0)
t_inside_all = torch.cat((t_inside,observe_data[:,1:2]),axis=0)

In [ ]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
test_loss_1 = []
C1_list = []
############## Record list ###############


nIter1 = 20000
it = 0

while it<nIter1:
    #########loss f#########
    loss_f = get_loss_f(x_inside_all,t_inside_all,PINNs1,C1,return_sequence='not')
    
    #########loss f  for collocation data#########
    loss_f_for_collocation = get_loss_f(x_inside,t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  for observation data#########
    loss_f_for_T = get_loss_f(observe_data_x_inside,observe_data_t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  excapt observation data#########
    loss_f_excapt_T = get_loss_f(test_data_x_inside,test_data_t_inside,PINNs1,C1,return_sequence='not')
    
    #########loss T observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation)
    loss_observation = torch.mean(torch.square(E_observation-observe_u))            
    
    #########loss T noisy observation#########  
    loss_observation_noisy = torch.mean(torch.square(E_observation-observe_u)) 
    
    #########loss T clear observation#########        
    E_observation_clear = PINNs1(observe_data) 
    E_observation_clear = output_transform(observe_data,E_observation_clear) 
    loss_observation_clear = torch.mean(torch.square(E_observation_clear-observe_clear_u))        
    
    #########loss T excapt observation#########        
    E_observation_excapt = PINNs1(test_data) 
    E_observation_excapt = output_transform(test_data,E_observation_excapt)
    loss_observation_excapt = torch.mean(torch.square(E_observation_excapt-test_u))      

    #########loss PI#########
    loss = loss_f+10*loss_observation
    
    #########test_loss NRMSE#########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data,pre_u)
    test_loss = relative_l2_torch(pre_u,test_u)
    #########test_loss NRMSE#########
    
    #########Record#########
    loss_f_1.append(loss_f.item())
    loss_f_for_collocation_1.append(loss_f_for_collocation.item())
    loss_f_for_T_1.append(loss_f_for_T.item())
    loss_f_excapt_T_1.append(loss_f_excapt_T.item())
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation_noisy.item()) 
    loss_T_clear_1.append(loss_observation_clear.item()) 
    loss_T_1_test_data.append(loss_observation_excapt.item()) 
    test_loss_1.append(test_loss)
    #########Record#########
    
    if it % 1000 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        print(C1)
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    
    it = it + 1   
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)